## 4.1 PostgreSQL - Approche relationnelle-objet

In [1]:
import psycopg2
import psycopg2.extras

conn = psycopg2.connect(
    host="localhost",
    database="demo",
    user="demo_owner",
    password="OODBMS")

# query result as dict
c = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

<div id="constructor" style="font-size:120%">
Exemple de constructeur
</div>

In [2]:
c.execute('DROP FUNCTION IF EXISTS new_student();')

c.execute('''
CREATE FUNCTION new_student() RETURNS students AS $$
    INSERT INTO students(person,address) VALUES (NULL,NULL)
        RETURNING *;
$$ LANGUAGE SQL;
''')
conn.commit()

In [28]:
c.execute('DELETE FROM students WHERE id > 3')
conn.commit()

c.execute('SELECT * FROM new_student()')
student_data = dict(c.fetchone())
print(student_data)

{'id': 41, 'person': None, 'address': None}


In [29]:
c.execute('SELECT * FROM students')
for s in c.fetchall(): print(s)

[1, '(Raymond,Deubaze)', '(31,"avenue Guy de Collongue",69130,Ecully)']
[2, '(Anna,Conda)', '(51,"chemin des Mouilles",69130,Ecully)']
[3, '(Jean,Peuplu)', '(51,"chemin des Mouilles",69130,Ecully)']
[41, None, None]


<div id="setters" style="font-size:120%">
Exemples de setters
</div>

In [30]:
# Setter pour la personne
c.execute('DROP FUNCTION IF EXISTS student_set_person(INT, TEXT, TEXT);')

c.execute('''
CREATE FUNCTION student_set_person(INTEGER, TEXT, TEXT) RETURNS students AS $$
    UPDATE students SET person.first_name=$2, person.last_name=$3 WHERE id = $1
        RETURNING *;
$$ LANGUAGE SQL;
''')
conn.commit()

In [31]:
c.execute("SELECT * FROM student_set_person(%s,'Alex','Terrieur')",(student_data['id'],))
student_data = dict(c.fetchone())
print(student_data)

{'id': 41, 'person': '(Alex,Terrieur)', 'address': None}


In [7]:
c.execute('SELECT * FROM students')
for s in c.fetchall(): print(s)

[1, '(Raymond,Deubaze)', '(31,"avenue Guy de Collongue",69130,Ecully)']
[2, '(Anna,Conda)', '(51,"chemin des Mouilles",69130,Ecully)']
[3, '(Jean,Peuplu)', '(51,"chemin des Mouilles",69130,Ecully)']
[39, '(Alex,Terrieur)', None]


In [8]:
# Setter pour l'adresse
c.execute('DROP FUNCTION IF EXISTS student_set_address(INTEGER, INTEGER, TEXT, INTEGER, TEXT);')

c.execute('''
CREATE FUNCTION student_set_address(INTEGER, INTEGER, TEXT, INTEGER, TEXT) RETURNS students AS $$
    UPDATE students SET address.number=$2, address.street=$3, address.zipcode=$4, address.city=$5
    WHERE id = $1
        RETURNING *;
$$ LANGUAGE SQL;
''')
conn.commit()

In [32]:
c.execute("SELECT * FROM student_set_address(%s,78,'route de Paris',69260,'Charbonnières-les-Bains')",(student_data['id'],))
student_data = dict(c.fetchone())
print(student_data)

{'id': 41, 'person': '(Alex,Terrieur)', 'address': '(78,"route de Paris",69260,Charbonnières-les-Bains)'}


In [33]:
c.execute('SELECT * FROM students')
for s in c.fetchall(): print(s)

[1, '(Raymond,Deubaze)', '(31,"avenue Guy de Collongue",69130,Ecully)']
[2, '(Anna,Conda)', '(51,"chemin des Mouilles",69130,Ecully)']
[3, '(Jean,Peuplu)', '(51,"chemin des Mouilles",69130,Ecully)']
[41, '(Alex,Terrieur)', '(78,"route de Paris",69260,Charbonnières-les-Bains)']


<div id="getters" style="font-size:120%">
Exemples de getters
</div>

In [11]:
c.execute('DROP FUNCTION IF EXISTS student_get_person(INTEGER);')

c.execute('''
CREATE FUNCTION student_get_person(INTEGER) RETURNS person_type AS $$
    SELECT person FROM students WHERE id = $1;
$$ LANGUAGE SQL;
''')
conn.commit()

In [12]:
c.execute("SELECT * FROM student_get_person(%s)",(student_data['id'],))
person = dict(c.fetchone())
print(person)

{'first_name': 'Alex', 'last_name': 'Terrieur'}


In [13]:
c.execute('DROP FUNCTION IF EXISTS student_get_address(INTEGER);')

c.execute('''
CREATE FUNCTION student_get_address(INTEGER) RETURNS address_type AS $$
    SELECT address FROM students WHERE id = $1;
$$ LANGUAGE SQL;
''')
conn.commit()

In [14]:
c.execute("SELECT * FROM student_get_address(%s)",(student_data['id'],))
address = dict(c.fetchone())
print(address)

{'number': 78, 'street': 'route de Paris', 'zipcode': 69260, 'city': 'Charbonnières-les-Bains'}


<div id="find" style="font-size:120%">
Fonctions de recherche
</div>

In [15]:
c.execute('''
DROP FUNCTION IF EXISTS student_find_by_id(INTEGER);

CREATE FUNCTION student_find_by_id(INTEGER) RETURNS students AS $$
    SELECT * FROM students WHERE id = $1;
$$ LANGUAGE SQL;

DROP FUNCTION IF EXISTS students_find_by_person(person_type);

CREATE FUNCTION students_find_by_person(person_type) RETURNS SETOF students AS $$
    SELECT * FROM students WHERE (person).first_name = ($1).first_name AND (person).last_name=($1).last_name;
$$ LANGUAGE SQL;

''')
conn.commit()

In [16]:
c.execute("SELECT * FROM student_find_by_id(%s)",(1,))
print(dict(c.fetchone()))

{'id': 1, 'person': '(Raymond,Deubaze)', 'address': '(31,"avenue Guy de Collongue",69130,Ecully)'}


In [17]:
c.execute("SELECT * FROM student_find_by_id(%s)",(33,))
print(dict(c.fetchone()))

{'id': None, 'person': None, 'address': None}


In [18]:
c.execute("SELECT * FROM students_find_by_person(%s)",(student_data['person'],))
for s in c.fetchall(): print(s)

[39, '(Alex,Terrieur)', '(78,"route de Paris",69260,Charbonnières-les-Bains)']


In [19]:
c.execute("SELECT * FROM students_find_by_person(%s)",(('Raymond','Deubaze'),))
for s in c.fetchall(): print(s)

[1, '(Raymond,Deubaze)', '(31,"avenue Guy de Collongue",69130,Ecully)']


In [20]:
c.execute("SELECT * FROM students_find_by_person(%s)",(('Alain','Terrieur'),))
for s in c.fetchall(): print(s)

<div id="delete" style="font-size:120%">
Procédure de suppression
</div>

In [21]:
c.execute('''
DROP PROCEDURE IF EXISTS student_delete(INTEGER);

CREATE PROCEDURE student_delete(INTEGER) AS $$
    DELETE FROM students WHERE id = $1;
$$ LANGUAGE SQL;

''')
conn.commit()

<div id="class" style="font-size:120%">
Utilisation via une classe python
</div>

In [22]:
class Student():

    @classmethod
    def by_id(cls, conn,  id):
        c = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        c.execute('SELECT * FROM student_find_by_id(%s)',(id,))
        return cls(conn,dict(c.fetchone()))

    @classmethod
    def by_name(cls, conn, first_name, last_name):
        c = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        c.execute('SELECT * FROM students_find_by_person(%s)',((first_name,last_name),))
        return [ cls(conn,dict(u)) for u in c.fetchall()]
        
    @classmethod
    def new(cls, conn):
        c = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        c.execute('SELECT * FROM new_student()')
        self = cls(conn,dict(c.fetchone()))
        return self
        
    def __init__(self,conn,student_data):
        self.__conn = conn
        for k in student_data:
            setattr(self,k,student_data[k])
        
    def set_person(self,first_name,last_name):
        c = self.__conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        c.execute('SELECT * FROM student_set_person(%s,%s,%s)',(self.id,first_name,last_name))
        self.person = dict(c.fetchone())['person']
        
    def set_address(self,number,street,zipcode,city):
        c = self.__conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        c.execute('SELECT * FROM student_set_address(%s,%s,%s,%s,%s)',(self.id,number,street,zipcode,city))
        self.address = dict(c.fetchone())['address']

    def delete(self):
        c = self.__conn.cursor()
        c.execute('CALL student_delete(%s)',(self.id,))
    
    def __str__(self):
        return '[{}] {} {}'.format(self.id, self.person,self.address)

In [23]:
student = Student.new(conn)
print(student)

student.set_person('Elsa','Plique')
print(student)

student.set_address(36,'Quai des Orfèvres',75001,'Paris')
print(student)

other = Student.by_id(conn,student.id)
print(other)

[40] None None
[40] (Elsa,Plique) None
[40] (Elsa,Plique) (36,"Quai des Orfèvres",75001,Paris)
[40] (Elsa,Plique) (36,"Quai des Orfèvres",75001,Paris)


In [24]:
c.execute("SELECT * FROM students")
for s in c.fetchall(): print(s)

[1, '(Raymond,Deubaze)', '(31,"avenue Guy de Collongue",69130,Ecully)']
[2, '(Anna,Conda)', '(51,"chemin des Mouilles",69130,Ecully)']
[3, '(Jean,Peuplu)', '(51,"chemin des Mouilles",69130,Ecully)']
[39, '(Alex,Terrieur)', '(78,"route de Paris",69260,Charbonnières-les-Bains)']
[40, '(Elsa,Plique)', '(36,"Quai des Orfèvres",75001,Paris)']


In [25]:
students = Student.by_name(conn,'Elsa','Plique')
for s in students: print(s)

[40] (Elsa,Plique) (36,"Quai des Orfèvres",75001,Paris)


In [26]:
for s in students: s.delete()

Student.by_name(conn,'Alex','Terrieur')[0].delete()

In [27]:
c.execute("SELECT * FROM students")
for s in c.fetchall(): print(s)

[1, '(Raymond,Deubaze)', '(31,"avenue Guy de Collongue",69130,Ecully)']
[2, '(Anna,Conda)', '(51,"chemin des Mouilles",69130,Ecully)']
[3, '(Jean,Peuplu)', '(51,"chemin des Mouilles",69130,Ecully)']


In [28]:
conn.close()